In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "4"

In [2]:
from aana.sdk import AanaSDK


aana_app = AanaSDK()
aana_app.connect(show_logs=True)

/root/.cache/pypoetry/virtualenvs/aana-vIr3-B0u-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-21 09:16:23,565	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/root/.cache/pypoetry/virtualenvs/aana-vIr3-B0u-py3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_dir" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ('settings_',)`.
  warnings.warn(
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> b5a993b53e6c, Initialize.
INFO  [al

(raylet) [2024-06-21 09:16:40,244 E 2194 2232] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-21_09-16-27_387283_1852 is over 95% full, available space: 8580161536; capacity: 983347249152. Object creation will fail if spilling is required.
(raylet) [2024-06-21 09:16:50,257 E 2194 2232] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-21_09-16-27_387283_1852 is over 95% full, available space: 8580141056; capacity: 983347249152. Object creation will fail if spilling is required.
(raylet) [2024-06-21 09:17:00,270 E 2194 2232] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-21_09-16-27_387283_1852 is over 95% full, available space: 8580005888; capacity: 983347249152. Object creation will fail if spilling is required.
(raylet) [2024-06-21 09:17:10,283 E 2194 2232] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-21_09-16-27_387283_1852 is over 95% full, available space: 8579870720; capacity: 983347249152. Object creation will fail if sp

In [4]:
from aana.deployments.face_detection_deployment import (
    FaceDetectorConfig,
    FaceDetectorDeployment,
)

face_detector_deployment = FaceDetectorDeployment.options(
    num_replicas=1,
    ray_actor_options={"num_gpus": 0.1},
    user_config=FaceDetectorConfig(
        model_url="/nas/dominic/Production/FaceModel_REST/InsightFace-REST/models/onnx/scrfd_10g_gnkps/scrfd_10g_gnkps.onnx",
        nms_thresh=0.4,
        batch_size=4,
        input_size=640,
    ).model_dump(mode="json"),
)

aana_app.register_deployment("face_detector", face_detector_deployment, deploy=True)

(ServeReplica:face_detector:FaceDetectorDeployment pid=7913) Available providers: ['CUDAExecutionProvider', 'CPUExecutionProvider']


(raylet) [2024-06-21 09:19:00,419 E 2194 2232] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-21_09-16-27_387283_1852 is over 95% full, available space: 7770275840; capacity: 983347249152. Object creation will fail if spilling is required.
(raylet) [2024-06-21 09:19:10,431 E 2194 2232] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-21_09-16-27_387283_1852 is over 95% full, available space: 7770267648; capacity: 983347249152. Object creation will fail if spilling is required.
(ServeReplica:face_detector:FaceDetectorDeployment pid=7913) INFO 2024-06-21 09:19:17,104 face_detector_FaceDetectorDeployment wzb9lrek 035c94fb-067d-47c6-bc16-da8da7329655 replica.py:373 - GET_METHODS OK 6.4ms
(raylet) [2024-06-21 09:19:20,444 E 2194 2232] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-06-21_09-16-27_387283_1852 is over 95% full, available space: 7770247168; capacity: 983347249152. Object creation will fail if spilling is required.
(ServeReplica:face_detec

In [5]:
from aana.deployments.aana_deployment_handle import AanaDeploymentHandle


handle = await AanaDeploymentHandle.create("face_detector")

In [6]:
from aana.core.models.image import Image
from pathlib import Path
image = Image(path=Path("/nas/dominic/Production/FaceModel_REST/InsightFace-REST/src/api_trt/test_images/group_1.jpg"))

In [7]:
detector_output = await handle.predict([image])

In [8]:
print(detector_output)

{'bounding_boxes': [array([[384.54086   ,  60.97224   , 447.36746   , 147.87277   ,
          0.8577642 ],
       [433.33582   , 143.24683   , 528.5499    , 251.911     ,
          0.8136517 ],
       [294.98123   ,  76.552765  , 369.68015   , 158.42337   ,
          0.8026969 ],
       [ 74.67878   , 174.83174   , 157.06747   , 287.8034    ,
          0.8024849 ],
       [240.47298   , 160.65451   , 310.59863   , 261.87784   ,
          0.7807217 ],
       [310.00235   , 160.83907   , 379.83856   , 261.0729    ,
          0.7789517 ],
       [166.35059   ,  83.13015   , 241.7204    , 178.81956   ,
          0.75995415]], dtype=float32)], 'keypoints': [array([[[395.08386 ,  97.8154  ],
        [425.6294  ,  90.19366 ],
        [410.0098  , 104.62544 ],
        [401.13202 , 124.9643  ],
        [429.13327 , 118.493324]],

       [[452.32224 , 193.64856 ],
        [489.60065 , 174.51643 ],
        [476.46063 , 206.2497  ],
        [469.32574 , 225.71524 ],
        [507.31046 , 208.23306 

In [10]:
from aana.deployments.face_featureextraction_deployment import (
    FacefeatureExtractorConfig,
    FacefeatureExtractorDeployment,
)

facefeat_extractor_deployment = FacefeatureExtractorDeployment.options(
    num_replicas=1,
    ray_actor_options={"num_gpus": 0.5},
    user_config=FacefeatureExtractorConfig(
        model_url="/nas/dominic/AdaFace/pretrained/adaface_ir50_webface4m.ckpt",
        min_face_norm=18.0
    ).model_dump(mode="json"),
)

aana_app.register_deployment(
    "facefeat_extractor", facefeat_extractor_deployment, deploy=True
)

In [11]:
from aana.deployments.aana_deployment_handle import AanaDeploymentHandle


facefeat_handle = await AanaDeploymentHandle.create("facefeat_extractor")

In [12]:
keypoints = detector_output['keypoints']

In [13]:
facefeat_output = await facefeat_handle.predict([image], keypoints)

In [14]:
facefeat_output

{'facefeats_per_image': [{'face_feats': array([[-0.01961349, -0.0623821 , -0.00813501, ..., -0.05425359,
            0.02324992, -0.01696939],
          [-0.04928799,  0.03148894, -0.03748646, ...,  0.09751447,
           -0.00310521,  0.04418212],
          [ 0.03298211,  0.00803477,  0.03870542, ..., -0.08577755,
           -0.05855932, -0.03082386],
          ...,
          [ 0.03602575, -0.06101663, -0.00808396, ...,  0.07198863,
           -0.06150961, -0.01345934],
          [-0.0419326 ,  0.00321984, -0.09398558, ...,  0.01119729,
           -0.02694403, -0.01147183],
          [-0.03697017,  0.00075135, -0.00875726, ...,  0.00301811,
            0.07731081,  0.00929155]], dtype=float32),
   'norms': array([[22.930727],
          [22.830807],
          [23.095766],
          [24.813797],
          [21.567142],
          [22.338127],
          [25.992365]], dtype=float32)}]}

In [20]:
len(facefeat_output['facefeats_per_image'][0]['face_feats'])

7

In [45]:
# Sample image paths
image_paths = [
    'image1.jpg', 'image2.jpg', 'image3.jpg', 'image4.jpg', 'image5.jpg',
    'image6.jpg', 'image7.jpg', 'image8.jpg', 'image9.jpg', 'image10.jpg'
]

# Populate the data dictionary
data = {
    "images": [{'path': im_path} for im_path in image_paths[0:10]]
}

# Print the result
print(data)

{'images': [{'path': 'image1.jpg'}, {'path': 'image2.jpg'}, {'path': 'image3.jpg'}, {'path': 'image4.jpg'}, {'path': 'image5.jpg'}, {'path': 'image6.jpg'}, {'path': 'image7.jpg'}, {'path': 'image8.jpg'}, {'path': 'image9.jpg'}, {'path': 'image10.jpg'}]}


In [49]:
import requests, json



def extract_face_feats(im_path):
    data = {
    "images": [
        {'path': im_path},
        {'path': im_path}

    ],
    }

    response = requests.post(
        "http://127.0.0.1:8000/recognize_faces",
        data={"body": json.dumps(data)},
        stream=False,
    )
    res = response.json()

    return res
    #return res['face_features_per_image'][0]['face_feats'], res['face_features_per_image'][0]['norms'] 

def extract_face_feats_batched(im_paths):
    data = {
        "images": [{'path': im_path} for im_path in im_paths]
    }

    print(data)


    response = requests.post(
        "http://127.0.0.1:8000/recognize_faces",
        data={"body": json.dumps(data)},
        stream=False,
    )
    res = response.json()
    
    return res

    # face_feats = [res['face_features_per_image'][k]['face_feats'] for k in range(len(res['face_features_per_image']))]
    # norms = [res['face_features_per_image'][k]['norms'] for k in range(len(res['face_features_per_image']))]

    # return face_feats, norms



In [40]:
res = extract_face_feats("/nas/dominic/AanaFaceEval/identities_reference/default/000bb305-88c1-4490-977f-59e66b755cd4|||Ilse Neubauer.jpg")

In [55]:
data = {
        "images": [{'path': im_path} for im_path in image_paths[0:10]]
    }

print(data)

{'images': [{'path': '/nas/dominic/AanaFaceEval/identities_reference/default/000744f4-4131-44db-a569-ca211fa55a48|||Keita Machida.jpg'}, {'path': '/nas/dominic/AanaFaceEval/identities_reference/default/0007970d-a5d9-4791-bb47-9c52d4cf4473|||Caroleen Feeney.jpg'}, {'path': '/nas/dominic/AanaFaceEval/identities_reference/default/0009aaba-e0da-46c3-a557-72d318ce0bd8|||Ewa Fröling.jpg'}, {'path': '/nas/dominic/AanaFaceEval/identities_reference/default/000b66a3-5009-4513-aa13-535202c679b3|||Marielle Heller.jpg'}, {'path': '/nas/dominic/AanaFaceEval/identities_reference/default/000bb305-88c1-4490-977f-59e66b755cd4|||Ilse Neubauer.jpg'}, {'path': '/nas/dominic/AanaFaceEval/identities_reference/default/000d385b-ac12-41b0-9692-7abff6c5546a|||Michel Joelsas.jpg'}, {'path': '/nas/dominic/AanaFaceEval/identities_reference/default/00112729-6837-424c-bdf5-1709b9b38cba|||Patricia Hodge.jpg'}, {'path': '/nas/dominic/AanaFaceEval/identities_reference/default/00115272-41a7-4b22-bb5a-ea079aa2dea6|||Gilli

In [72]:
res = extract_face_feats_batched(image_paths[13:19])

{'images': [{'path': '/nas/dominic/AanaFaceEval/identities_reference/default/001c3bcf-3ab0-48cc-b805-c8da6ed3ba6a|||Eryk Lubos.jpg'}, {'path': '/nas/dominic/AanaFaceEval/identities_reference/default/001d0484-4b83-42da-bff9-145bcc46359b|||Sanjjanaa Archana Galrani.jpg'}, {'path': '/nas/dominic/AanaFaceEval/identities_reference/default/001e04f1-a8f4-45e0-a85d-59d8bf06f12a|||Gerald Okamura.jpg'}, {'path': '/nas/dominic/AanaFaceEval/identities_reference/default/0021297f-e5f7-4965-a0cd-bb42c4d70732|||Laurie Metcalf.jpg'}, {'path': '/nas/dominic/AanaFaceEval/identities_reference/default/00227c4a-8cf5-491f-823d-4bd6251efac1|||Luís Melo.jpg'}, {'path': '/nas/dominic/AanaFaceEval/identities_reference/default/0022e2e0-f72b-4470-af26-0b93527b3ebf|||Marie-Mae van Zuilen.jpg'}]}


In [73]:
res

{'error': 'IndexError',
 'message': 'list index out of range',
 'data': {},
 'stacktrace': '\x1bray::ServeReplica:facefeat_extractor_deployment:FacefeatureExtractorDeployment.handle_request_with_rejection()\x1b (pid=9498, ip=172.17.0.3, actor_id=2d13ca81aac0a71ca1ac728801000000, repr=<ray.serve._private.replica.ServeReplica:facefeat_extractor_deployment:FacefeatureExtractorDeployment object at 0x7f035bcd8af0>)\n    yield await self._user_callable_wrapper.call_user_method(\n  File "/root/.cache/pypoetry/virtualenvs/aana-vIr3-B0u-py3.10/lib/python3.10/site-packages/ray/serve/_private/replica.py", line 1150, in call_user_method\n    raise e from None\nray.exceptions.RayTaskError: \x1bray::ServeReplica:facefeat_extractor_deployment:FacefeatureExtractorDeployment.handle_request_with_rejection()\x1b (pid=9498, ip=172.17.0.3)\n  File "/root/.cache/pypoetry/virtualenvs/aana-vIr3-B0u-py3.10/lib/python3.10/site-packages/ray/serve/_private/utils.py", line 168, in wrap_to_ray_error\n    raise exce

In [2]:
import tqdm
import glob
import os


reference_faces_dict_30K_webface4M_r101 = {'person_names': [],
                       'paths_to_image': [],
                       'face_features': [],
                           'norms': []}

failed_faces = []

image_paths = sorted(glob.glob('/nas/dominic/AanaFaceEval/identities_reference/default/*.jpg'))

for image_filepath in tqdm.tqdm(image_paths):

    face_feats, norms = extract_face_feats(image_filepath)

    #import pdb; pdb.set_trace()

    if(face_feats is not None and len(face_feats)==1 and norms[0][0]>=19.0):

        reference_faces_dict_30K_webface4M_r101['person_names'].append(os.path.basename(image_filepath)[0:-4].split('|||')[-1])
        reference_faces_dict_30K_webface4M_r101['paths_to_image'].append(image_filepath)
        reference_faces_dict_30K_webface4M_r101['face_features'].append(face_feats[0])
        reference_faces_dict_30K_webface4M_r101['norms'].append(norms[0][0])
    else:
        failed_faces.append([image_filepath, norms[0][0]])

reverse_reference_faces_dict_30K_webface4M_r101 = {reference_faces_dict_30K_webface4M_r101['person_names'][k]: reference_faces_dict_30K_webface4M_r101['paths_to_image'][k] for k in range(len(reference_faces_dict_30K_webface4M_r101['person_names']))}

  0%|          | 28/33541 [00:07<2:38:11,  3.53it/s]


KeyboardInterrupt: 